In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# 요청 헤더
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Authorization': 'Bearer null',
    'Content-Type': 'application/json;charset=UTF-8',
    'Cookie': '.AspNetCore.Session=CfDJ8KmmMWsifXpFnv2%2BiN8%2Fp1dpvk8kyaL65%2Bl0BlaLfKUHqhAuPlL1Sx1eghQAm3I3hawHj%2BxHtwN%2FqQ%2B9pmfMzZPXq1rUo3awOUINi4ulSgVdzYDvFOpuDu4f1n6frqzF7LgPWIA%2FdZVMah9AP34jgYZDv569RE6S3FqUqSRhtpau',
    'Host': 'kcsc.re.kr',
    'Origin': 'https://kcsc.re.kr',
    'Referer': 'https://kcsc.re.kr/StandardCode/Viewer/41024',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Whale/3.25.232.19 Safari/537.36'
}

# 요청 데이터
data = {}

# POST 요청 보내기
url = 'https://kcsc.re.kr/api/StandardCode/All/List'
response = requests.post(url, headers=headers, json=data)

# 응답 확인
if response.status_code == 200:
    # JSON 형식으로 받은 데이터를 BeautifulSoup을 사용하여 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    # JSON 문자열 추출
    json_str = soup.get_text()
    # JSON 문자열을 파이썬 딕셔너리로 변환
    result_data = json.loads(json_str)

    # 데이터 추출 및 처리
    seq_list = []
    code_type_list = []
    code_version_list = []
    code_name_list = []
    parent_seq_list = []
    code_sort_list = []

    for item in result_data["list"]:
        seq_list.append(item["seq"])
        code_type_list.append(item["codeType"])
        code_version_list.append(item["codeVersion"])
        code_name_list.append(item["codeName"])
        parent_seq_list.append(item["parentSeq"])
        code_sort_list.append(item["codeSort"])

    # 데이터프레임으로 변환
    df = pd.DataFrame({
        "seq": seq_list,
        "code_type": code_type_list,
        "code_version": code_version_list,
        "code_name": code_name_list,
        "parent_seq": parent_seq_list,
        "code_sort": code_sort_list
    })

    print(df)

else:
    print("오류 발생:", response.status_code)


        seq code_type code_version     code_name  parent_seq  code_sort
0         2        10         None          설계기준         NaN       5241
1         4        10         None           공통편         2.0        176
2         7        10         None       공통 설계기준         4.0        177
3     41250        10         2021        공통설계기준         7.0          9
4     40192        10         2021       설계기준 총칙         7.0         73
...     ...       ...          ...           ...         ...        ...
4272  40435        90         2021      수도미터 보호통     40257.0        971
4273  40436        90         2021     상수도공사 압력계     40238.0        973
4274  40437        90         2021  상수도공사 수질측정장치     40238.0        974
4275  40438        90         2021  상수도공사 수질감시장치     40238.0        975
4276  40439        90         2021  상수도공사 전광판 설치     40238.0       6192

[4277 rows x 6 columns]


In [3]:
import requests
import pandas as pd
import time 

# 요청 헤더
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Authorization': 'Bearer null',
    'Content-Type': 'application/json;charset=UTF-8',
    'Cookie': '.AspNetCore.Session=CfDJ8KmmMWsifXpFnv2%2BiN8%2Fp1dpvk8kyaL65%2Bl0BlaLfKUHqhAuPlL1Sx1eghQAm3I3hawHj%2BxHtwN%2FqQ%2B9pmfMzZPXq1rUo3awOUINi4ulSgVdzYDvFOpuDu4f1n6frqzF7LgPWIA%2FdZVMah9AP34jgYZDv569RE6S3FqUqSRhtpau',
    'Host': 'kcsc.re.kr',
    'Origin': 'https://kcsc.re.kr',
    'Referer': 'https://kcsc.re.kr/StandardCode/Viewer/{}',  # {} 부분이 seq로 대체될 예정
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Whale/3.25.232.19 Safari/537.36'
}

# 요청 데이터
data = {}

# 결과를 저장할 리스트 초기화
contents_title_list = []
sub_view_list = []
code_name_list = []
seq_list = []

# 모든 seq에 대해 반복
for seq_index, seq_value in enumerate(df["seq"], start=1):
    #혹시 모르니 4초 웨이팅 추가
    time.sleep(4)
    # 요청 헤더에서 Referer 부분을 seq로 대체
    headers['Referer'] = headers['Referer'].format(seq_value)

    # 요청 데이터 설정
    data["paramSeq"] = seq_value

    # POST 요청 보내기
    url = 'https://kcsc.re.kr/api/ViewerApi/Detail'
    response = requests.post(url, headers=headers, json=data)

    # 응답 확인
    if response.status_code == 200:
        # JSON 형식으로 받은 데이터를 딕셔너리로 변환
        result_data = response.json()

        # 모든 documents의 정보를 가져와서 리스트에 추가
        documents = result_data["detail"]["listDocument"]
        for document in documents:
            contents_title_list.append(document["contentsTitle"])
            sub_view_list.append(document["subView"])
            code_name_list.append(result_data["detail"]["codeName"])
            seq_list.append(result_data["paramSeq"])

        # 중간 결과를 DataFrame에 추가 및 저장 (100개씩 저장)
        if seq_index % 100 == 0 or seq_index == len(df["seq"]):
            temp_df = pd.DataFrame({
                "seq": seq_list,
                "codeName": code_name_list,
                "contentsTitle": contents_title_list,
                "subView": sub_view_list
            })
            temp_df.to_csv(f"df_intermediate_{seq_index}.csv", index=False)  # CSV 파일로 저장

            # 저장 후 리스트 초기화
            contents_title_list = []
            sub_view_list = []
            code_name_list = []
            seq_list = []

    else:
        print(f"오류 발생: {response.status_code} - seq: {seq_value}")

# 최종 결과 리스트를 데이터프레임에 추가
df_final = pd.DataFrame({
    "seq": seq_list,
    "codeName": code_name_list,
    "contentsTitle": contents_title_list,
    "subView": sub_view_list
})

# 최종 결과 DataFrame을 CSV 파일로 저장
df_final.to_csv("final_df.csv", index=False)

print("작업 완료")


작업 완료
